In [1]:
import os
import time
import copy
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
from utils_torch import *

In [2]:
'''Load training, validation and test data'''

batch_size = 32
input_shape = (192, 192, 3)
train_ids = pd.read_csv('csv_file/annot_training_set.csv')
val_ids = pd.read_csv('csv_file/annot_val_set.csv')
test_ids = pd.read_csv('csv_file/annot_test_set.csv')
print('training data:', len(train_ids))
print('valid data:',len(val_ids))
print('test data:',len(test_ids))

dataset_sizes = {'train':len(train_ids),'val':len(val_ids),'test':len(test_ids)}
print(dataset_sizes)


training data: 199603
valid data: 29245
test data: 54529
{'train': 199603, 'val': 29245, 'test': 54529}


In [3]:
'''Load labels for multi-class classification'''

names_to_labels = {}
df_class = pd.read_csv('36_class.csv',header=None)
for index, row in df_class.iterrows():
    names_to_labels[row[0]] = row[1]

print(len(names_to_labels))

36


In [4]:
train_set = HelmetDataset(ids=train_ids,
                          root_dir='Myanmar_data/training_set/image/',
                          names_to_labels = names_to_labels,
                          transform=transforms.Compose([
                              transforms.RandomRotation(10),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((192,192)),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            ]))


val_set = HelmetDataset(ids=val_ids,
                        root_dir='Myanmar_data/val_set/image/',
                        names_to_labels = names_to_labels,
                        transform=transforms.Compose([
                            transforms.Resize((192,192)),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            ]))

test_set = HelmetDataset(ids=test_ids,
                         root_dir='Myanmar_data/test_set/image/',
                         names_to_labels = names_to_labels,
                         transform=transforms.Compose([
                             transforms.Resize((192,192)),
                             transforms.ToTensor(),
                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                            ]))

dataloaders = {'train':DataLoader(train_set, batch_size=batch_size,shuffle=True, num_workers=4)
               ,'val':DataLoader(val_set, batch_size=batch_size,shuffle=False, num_workers=4)
               ,'test':DataLoader(test_set, batch_size=batch_size,shuffle=False, num_workers=4)}
print(dataloaders)



device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

{'train': <torch.utils.data.dataloader.DataLoader object at 0x7fd61c5a22d0>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7fd61c5a24d0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7fd61c5a2590>}


In [5]:
def train_model(model, optimizer, scheduler, num_epochs=1):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    #best_loss = np.Inf
    best_acc = 0
    epoch_ACCs = []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.   
            for i_batch, sample_batched in tqdm(enumerate(dataloaders[phase])):
                
                inputs, labels = sample_batched['image'], sample_batched['label']
                inputs, labels = inputs.type(torch.cuda.FloatTensor), labels.type(torch.cuda.LongTensor)
                inputs, labels = inputs.to(device), labels.to(device)
                
                
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #print(preds.shape,labels.shape)
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                                   
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_ACCs.append(epoch_acc.item())

            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val loss: {:4f}'.format(best_loss))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_ACCs

In [6]:
'''Load pretrained model'''

model_ft = models.resnet34(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,len(names_to_labels))

#print(model_ft)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001)
optimizer_ft = optim.Adam(model_ft.parameters(),lr=0.0001)

# Decay LR by a factor of 0.1 every 10 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [7]:
savepath = 'save_model/best-cross-entropy'
model_ft, epoch_ACCs = train_model(model_ft, optimizer_ft, exp_lr_scheduler, num_epochs=10)
torch.save(model_ft.state_dict(),savepath)

Epoch 1/10
----------


6238it [16:44,  6.21it/s]

train Loss: 0.4776 Acc: 0.8513



914it [02:24,  6.34it/s]

val Loss: 0.8637 Acc: 0.7944

Epoch 2/10
----------


...

val Loss: 1.1321 Acc: 0.8258

Epoch 10/10
----------



6238it [16:46,  6.20it/s]

train Loss: 0.0120 Acc: 0.9960



914it [02:23,  6.36it/s]


val Loss: 1.1864 Acc: 0.8227

Training complete in 217m 12s
Best val Acc: 0.828689


In [7]:
savepath = 'save_model/best-cross-entropy'
model_ft.load_state_dict(torch.load(savepath))
model_ft.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
'''Evaluation on test set'''
phase = 'test'
running_loss = 0.0
running_corrects = 0
y_pred = []
for i_batch, sample_batched in enumerate(dataloaders[phase]):
    inputs, labels = sample_batched['image'], sample_batched['label']
    inputs, labels = inputs.type(torch.cuda.FloatTensor), labels.type(torch.cuda.LongTensor)
    inputs, labels = inputs.to(device), labels.to(device)
    
    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    y_pred.append(preds.cpu().numpy())
    loss = criterion(outputs, labels)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    
    if i_batch % 50 == 49:
        print('[%5d] loss: %.3f' %(i_batch + 1, running_loss / (i_batch*20)))
                
    
epoch_loss = running_loss / dataset_sizes[phase]
epoch_acc = running_corrects.double() / dataset_sizes[phase]

print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

[   50] loss: 1.957
[  100] loss: 1.927
[  150] loss: 1.753
[  200] loss: 1.571
[  250] loss: 1.617
[  300] loss: 1.699
[  350] loss: 1.994
[  400] loss: 1.944
[  450] loss: 2.002
[  500] loss: 1.926
[  550] loss: 1.833
[  600] loss: 1.786
[  650] loss: 1.809
[  700] loss: 1.878
[  750] loss: 1.927
[  800] loss: 1.909
[  850] loss: 1.846
[  900] loss: 1.825
[  950] loss: 1.775
[ 1000] loss: 1.813
[ 1050] loss: 1.812
[ 1100] loss: 1.775
[ 1150] loss: 1.726
[ 1200] loss: 1.770
[ 1250] loss: 1.753
[ 1300] loss: 1.725
[ 1350] loss: 1.821
[ 1400] loss: 1.795
[ 1450] loss: 1.780
[ 1500] loss: 1.783
[ 1550] loss: 1.761
[ 1600] loss: 1.762
[ 1650] loss: 1.760
[ 1700] loss: 1.802
test Loss: 1.1232 Acc: 0.8228
